In [1]:
import numpy as np
import pandas as pd

In [51]:
import spacy

In [26]:
from spacy.tokenizer import Tokenizer
import nltk

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [22]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [29]:
from spacy.vocab import Vocab

In [6]:
df_train = pd.read_csv("c:/data/kaggle/Disaster tweets/train.csv")

In [12]:
df_train.isna().sum(axis=0)

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [21]:
df_train[df_train["keyword"].isnull()==False]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


We have three columns to use to predict on, keyword and text are nlp, location could help but leave for now

In [42]:
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [45]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [53]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [56]:
example = nlp(df_train["text"][1])

In [60]:
example.vector

array([ 1.0309792 , -1.2418449 , -0.70834446,  2.372108  ,  2.240989  ,
        2.5972116 , -0.01759425,  0.5596446 ,  1.4090172 ,  1.6113966 ,
        1.5328721 , -0.9112137 ,  0.47672936, -1.4448929 , -0.9597081 ,
       -1.3382363 ,  1.2704474 ,  1.4045588 ,  1.0939984 , -0.39732674,
       -0.25665942, -1.0874226 ,  1.1145511 , -1.4253209 , -0.56490976,
        0.07184009,  0.27531934, -0.9448942 ,  0.34717762, -1.9551909 ,
       -0.1868523 ,  0.5933187 ,  1.204904  ,  0.37316045,  0.93774116,
       -2.2168355 ,  3.102009  , -1.7005    , -0.8508695 , -0.5461243 ,
        1.9598846 , -0.17920429,  0.44159058, -2.3268094 , -0.7807982 ,
       -1.1765788 ,  0.90138495, -0.9002015 , -0.4945909 , -0.31053415,
       -0.46314704, -2.1820452 , -0.547442  , -0.58392787, -3.1788797 ,
        0.4143385 ,  0.5025425 ,  2.1572542 , -0.569477  ,  0.11471212,
       -0.52791154, -0.9969019 ,  0.43371034,  0.24727356, -0.5470768 ,
        0.6669565 ,  0.4905461 , -1.7539945 , -0.9240975 ,  1.54

In [62]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [72]:
spacy_tokenizer(df_train["text"][700])

['@bekah__w',
 'thanks',
 'sweat',
 'bullets',
 'time',
 'blazing',
 'sun',
 'beating']

In [73]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [34]:
tokens = Tokenizer(Vocab(df_train["text"]))

In [37]:
tfidf_matrix = TfidfVectorizer(tokens)

In [41]:
tfidf_matrix

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input=<spacy.tokenizer.Tokenizer object at 0x000001A7FD371A98>,
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
